In [46]:
! pip install --upgrade google-generativeai langchain-google-genai python-dotenv

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [48]:
from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
import google.generativeai as genai
import os
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

In [ ]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
import json
with open("a song of ice and fire.json", "r", encoding="utf-8") as file:
    data_dict = json.load(file)

In [ ]:

docs = ''.join(list(data_dict.values()))
# data = sum(docs.values())
# data

In [ ]:
#documents = [{"title": i, "content": }]

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.create_documents([docs])
len(texts)
texts[:5]

NameError: name 'RecursiveCharacterTextSplitter' is not defined

In [25]:
#simple test
test_text = "丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia"


In [26]:
query_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_document"
)
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(texts, embedding_function)
#test_text = "丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia"
#query_embeddings.embed_documents(texts)


NameError: name 'GoogleGenerativeAIEmbeddings' is not defined

In [27]:

query = "石头"
query_embeddings.embed_query(query)
retriever = db.as_retriever()
retriever.get("tyrion lannister")

NameError: name 'query_embeddings' is not defined

In [28]:
def make_qa_chain():
    llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
    return RetrievalQA.from_chain_type(
        llm,
        retriever=db.as_retriever(),
        return_source_documents=True
    )
qa_chain = make_qa_chain()

NameError: name 'ChatGoogleGenerativeAI' is not defined

In [29]:
q = "北境的领主有哪些？"
result = qa_chain({"query": q})
print(result["result"],"\nsources:",)
for idx, elt in enumerate(result["source_documents"]):
    print(elt.page_content)

NameError: name 'qa_chain' is not defined

In [30]:
1

1

In [33]:
print("asd")

asd
